In [ ]:
import os
import copy
import datasets as ds
import FPMLmodule.utils as utils
import tensorflow as tf
import FPMLmodule.backbones as backbones
import FPMLmodule.classifiers as classifiers
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from FPMLmodule.fpml import FPML 
from pathlib import Path

print('TensorFlow Version:', tf.__version__)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
weightsRN50 = "./weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightsMNV2 = "./weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"
weightsENB2 = "./weights/efficientnetb2_notop.h5"
weightsINCEPTIONV3 = "./weights/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightsXCEPTION = "./weights/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [ ]:
def ablation(configureDS, datasets, imgDim,epochsSearch, epochsBest, path, optimizers, learningRates, hypers, losses, backboneForTypeMode, verbose=1):
    trainDs, testDs, valDs = datasets
    nbClasses = len(configureDS.classNames)
    activation = "softmax"
    outFilePath = path + 'ablationHistory.csv'
    
    trainingModeStudy =  {
        'DNN-TransferLearningMode' : {
            "architecture": {
                "backbone": [
                    backboneForTypeMode(imgDim, weights=weightsMNV2, trainable=False, name=backboneForTypeMode.__name__+'PT&Untrainable')#,
                    # backboneForTypeMode(imgDim, weights=weightsMNV2, trainable=True, name=backboneForTypeMode.__name__+'PT&Trainable'),
                    # backboneForTypeMode(imgDim, weights=None, trainable=True, name=backboneForTypeMode.__name__+'IN&Trainable')
                    ],
                "classfier": classifiers.DefaultClassifier(nbClasses, activation),
                "inputLayer": None, 
                "inputDim": imgDim
            },
            "hyperparameters":hypers
        }
    }
    trainingModeHistories = utils.researchStudies(trainDs, valDs, trainingModeStudy, epochsSearch, verbose)
    trainingModeBestModel = utils.getBestStudyFromHistories(trainingModeHistories)
    trainable = 'Trainable' in trainingModeBestModel
    pretrained = 'PT' in trainingModeBestModel
    utils.saveStudyHistory(trainingModeStudy, trainingModeHistories, outFilePath)

    print("Training Mode Selected - Pretrained:", pretrained, 'Trainable:', trainable)

    # Backbone
    bestBackboneStudy = {
        "bestBackbone" : {
            "architecture": {
                "backbone": [
                    backbones.ResNet50(imgDim, weights=weightsRN50, trainable=trainable)#,
                    # backbones.EfficientNetB2(imgDim, weights=weightsENB2, trainable=trainable),
                    # backbones.InceptionV3(imgDim, weights=weightsINCEPTIONV3, trainable=trainable),
                    # backbones.Xception(imgDim, weights=weightsXCEPTION, trainable=trainable)
                    ],
                "classfier": classifiers.DefaultClassifier(nbClasses, activation),
                "inputLayer":"", 
                "inputDim": imgDim
            },
            "hyperparameters": hypers
        },
    }
    bestBackboneHistories = utils.researchStudies(trainDs, valDs, bestBackboneStudy, epochsSearch, verbose)
    bestBackboneHistories[trainingModeBestModel] = trainingModeHistories[trainingModeBestModel]
    bestBackbone = utils.getBestStudyFromHistories(bestBackboneHistories)
    utils.saveStudyHistory(bestBackboneStudy, bestBackboneHistories, outFilePath)
    print("Best DNN Backbon:", bestBackbone)


    # Optimizer
    optimizersStudy = copy.deepcopy(bestBackboneHistories[bestBackbone]['config'])
    optimizersStudy["hyperparameters"]["optimizer"] = optimizers[:1]
    optimizersHistories = utils.researchStudies(trainDs, valDs, {bestBackbone : optimizersStudy}, epochsSearch, verbose)
    optimizersHistories[bestBackbone+"_Adam"] = bestBackboneHistories[bestBackbone]
    bestOptimizier = utils.getBestStudyFromHistories(optimizersHistories)
    utils.saveStudyHistory({bestBackbone: optimizersStudy}, optimizersHistories, outFilePath)
    print("Best Optimizier:", bestOptimizier)


    # Learning rate
    learningRatesStudy = copy.deepcopy(optimizersHistories[bestOptimizier]['config'])
    learningRatesStudy['hyperparameters']['learningRate'] = learningRates[:1]
    learningRatesHistories = utils.researchStudies(trainDs, valDs, {bestOptimizier: learningRatesStudy}, epochsSearch, verbose)
    learningRatesHistories[bestOptimizier+"_"+str(hypers["learningRate"])] = optimizersHistories[bestOptimizier]
    bestLearningRate = utils.getBestStudyFromHistories(learningRatesHistories)
    utils.saveStudyHistory({bestOptimizier: learningRatesStudy}, learningRatesHistories, outFilePath)
    print("Best Learning Rate:", bestLearningRate)
    
    # loss function
    lossStudy = copy.deepcopy(learningRatesHistories[bestLearningRate]['config'])
    lossStudy['hyperparameters']["loss"] = losses[:1]
    lossesHistories = utils.researchStudies(trainDs, valDs, {bestLearningRate: lossStudy}, epochsSearch, verbose)
    lossesHistories[bestLearningRate+"_"+str(hypers["loss"])] = learningRatesHistories[bestLearningRate]
    bestLoss = utils.getBestStudyFromHistories(lossesHistories)
    utils.saveStudyHistory({bestLearningRate: lossStudy}, lossesHistories, outFilePath)
    print(bestLoss)

    bestModelName = bestLoss
    bestModelConfiguration = lossesHistories[bestLoss]['config']
    trainValDs = trainDs.concatenate(valDs)
    
    bestModel = FPML(**bestModelConfiguration["architecture"]).create(**bestModelConfiguration["hyperparameters"])
    bestModelHistory = bestModel.fit(trainValDs, validation_data=testDs, epochs=epochsBest, verbose=verbose)
    toSave = {'Best Model':{'history' : bestModelHistory, 'config' : bestModelConfiguration}}
    utils.saveStudyHistory({'Best Model': bestModelConfiguration}, toSave, outFilePath)
    bestModel.save('./weights/FPML_'+configureDS.name+'_'+bestModelName+".h5")
    

optimizers=[Nadam, RMSprop]
learningRates = [0.1, 0.01, 0.0001, 0.00001]
defaultHypers = {
        "optimizer": Adam,
        "learningRate": 0.001,
        "loss": 'binary_crossentropy',
        "metrics": 'accuracy'
    }
losses = ['binary_focal_crossentropy', 'hinge']
backboneForTypeMode = backbones.MobileNetV2



In [ ]:
# Global Config
seed=9
imgDim = (120, 120, 3)
imgHeight, imgWidth, imgChannels = imgDim
batchSize = 32
verbose = 1
outDir = './finalout/'

# Dataset configuration
dsConfig = {
    'batchSize': batchSize, 
    'parallelTune': tf.data.AUTOTUNE, 
    'split': [0.7, 0.15, 0.15], 
    'inputDim': (120, 120, 3), 
    'seed': seed, 
    'shuffle': True
    }



# Training interval
epochsForSearch = 1
epochsForBest = 2

In [ ]:


Dataset = ds.SOCOFingGender
configureDS = Dataset(**dsConfig, sampling=ds.SOCOFingGender.UNDER_SAMPLING)
datasets = configureDS.create()
dbStudyOutPath = outDir+'/'+configureDS.name+'/'
Path(dbStudyOutPath).mkdir(parents=True, exist_ok=True)
utils.datasetAnalysisAndDisplay(datasets, configureDS, dbStudyOutPath)
ablation(configureDS, datasets, imgDim, epochsForSearch, epochsForBest, dbStudyOutPath+'/', optimizers, learningRates, defaultHypers, losses, backboneForTypeMode, verbose=1)

In [ ]:


Dataset = ds.SOCOFingHeands
configureDS = Dataset(**dsConfig)
datasets = configureDS.create()
dbStudyOutPath = outDir+'/'+configureDS.name+'/'
Path(dbStudyOutPath).mkdir(parents=True, exist_ok=True)
utils.datasetAnalysisAndDisplay(datasets, configureDS, dbStudyOutPath)
ablation(configureDS, datasets, imgDim, epochsForSearch, epochsForBest, dbStudyOutPath+'/', optimizers, learningRates, defaultHypers, losses, backboneForTypeMode, verbose=1)